# Primal 4.0 - INCOMPLETE
Redone from the second section of Prof. Kruczenski's Maple script file `K_Psp.maple`, while accounting for a pole at threshold.

Remarks:
- At first glance, it looks like the data points generately using `scipy` alone are not **as accurate as** Maple, but seem to be **accurate enough** (hopefully).
- Implementing `sympy` will grant us the ability to manipulate the variables/data to an arbitrary precision, like Maple and Mathematica.
- In the case where the data generated using Python is pretty far off the more precise values generated in Maple/Mathematica, we will use Python solely for importing the data files and optimizing with `cvxpy`, without generating data in Python.

In [ ]:
import os
import numpy as np
from scipy.special import lpn
from scipy.special import lqn
import sympy as smp

# import scipy as sp
# import cvxpy as cp
# import matplotlib.pyplot as plt
# import mpmath as mp

## Computation of $A_{lj, mn}$, $B1$, $B2$ coefficients for single and double dispersion
### Basis Functions

$$
\begin{align*}
    \phi(n, \phi) & = \frac{2}{\sin^2 \left( \frac{\phi}{2} \right)} \sin(n \phi) \\
    % zf(s) & = \frac{2 - \sqrt{4 - s}}{2 + \sqrt{4 - s}} \\
    z_f(s) & = \frac{2 - \sqrt{4 - s}}{2 + \sqrt{4 - s}} \\
    \Phi(n, s) & = z_f^n(s) \\
    \Phi_t(n, \xi) & = \mathrm{e}^{i n \xi} \\
    \Phi_0(s) & = \frac{z_f(s)}{1 - z_f(s)} \\
    \Phi_{t0}(\xi) & = \frac{\mathrm{e}^{i \xi}}{1 - \mathrm{e}^{i \xi}} \\
    \Phi_f(n, s) & = \begin{cases}
                        \Phi_0(s), & \text{if } n = 0 \\
                        \Phi(n, s), & \text{otherwise}
                    \end{cases} \\
    t_1(\sigma) & = 4 ( 1 - \sigma^2 ) \\

    F_b(n, l, u, \sigma) & = \text{simplify}\left(-\frac{2}{u^2 - 4} \cdot \frac{d}{d\sigma}t_1(\sigma) \cdot \Phi(n, t_1(\sigma)) \cdot P_l\left(1 + \frac{2t_1(\sigma)}{u^2 - 4}\right)\right) \\
    F_{b0}(l, u, \sigma) & = \text{simplify}\left(-\frac{2}{u^2 - 4} \cdot \frac{d}{d\sigma}t_1(\sigma) \cdot \Phi_0(t_1(\sigma)) \cdot P_l\left(1 + \frac{2t_1(\sigma)}{u^2 - 4}\right)\right) \\
    \text{simplify}\left(\frac{\partial^2}{\partial s^2}\Phi(n, s)\right)
\end{align*}
$$


In [2]:
def delta(x, y):
    return 1 if x == y else 0


def phi(n, phi):
    return 2 / np.sin(phi / 2) ** 2 * np.sin(n * phi)


def zf(s):
    return (2 - np.sqrt(4 - s)) / (2 + np.sqrt(4 - s))


def Phi(n, s):
    return zf(s) ** n


def Phit(n, xi):
    return np.exp(1j * n * xi)


def Phi0(s):
    return zf(s) / (1 - zf(s))


def Phit0(xi):
    return np.exp(1j * xi) / (1 - np.exp(1j * xi))


def Phif(n, s):
    return Phi0(s) if n == 0 else Phi(n, s)


def t1(sigma):
    return 4 * (1 - sigma**2)


def Fb(n, l, u, sigma):
    t = t1(sigma)
    dt_dsigma = -8 * sigma
    leg_arg = 1 + 2 * t / (u**2 - 4)
    return -2 / (u**2 - 4) * dt_dsigma * Phi(n, t) * lpn(l, leg_arg)


def Fb0(l, u, sigma):
    t = t1(sigma)
    dt_dsigma = -8 * sigma
    leg_arg = 1 + 2 * t / (u**2 - 4)
    return -2 / (u**2 - 4) * dt_dsigma * Phi0(t) * lpn(l, leg_arg)

## Computation of $A_{lj, mn}$ coefficients with no pole in double dispersion and a pole in the single dispersion using one extra function in the basis

We take $V_{max} = [[8, 14, 32, 32, 14]]$ so we can loop over every array given (only 1 for now).

In [7]:
np.set_printoptions(precision=600)

########## VALUES ##########
# V = [[8, 14, 32, 32, 14]]
lmax = 8
Nmax = 14
M = 32
Lambda = 32
Nmax2 = 14

lmax2 = 2 * lmax - 1  # Maximum angular momentum to get lmax odd partial waves

dataFHFile = os.path.join(
    os.getcwd(),
    f"primal_4.0_points_data_P3_pole0_lmax{lmax}_Nmax{Nmax}_Nmax2{Nmax2}.dat",
)
dataMspFile = os.path.join(
    os.getcwd(),
    f"primal_4.0_points_data_Psp_K_pole0_lmax{lmax}_Nmax{Nmax}_M{M}_Lambda{Lambda}_Nmax2{Nmax2}.dat",
)
dataMspFileW = os.path.join(
    os.getcwd(),
    f"primal_4.0_points_data_Psp_W_pole0_lmax{lmax}_Nmax{Nmax}_M{M}_Lambda{Lambda}_Nmax2{Nmax2}.dat",
)

# xi = np.linspace(np.pi/M * (1/2), np.pi/M * (2*M - 1/2), 2*M)       # Interpolation points
# s = sa + (4 - sa) / np.cos(xi/2)**2                                 # Mapping from xi to s
# rho = np.pi/4 * np.sqrt((s - 4)/s)                                  # Mapping from s to rho


print("lmax =", lmax)
print("Nmax =", Nmax)
print("M =", M)
print("Lambda =", Lambda)
print("Nmax2 =", Nmax2)
print("lmax2 =", lmax2)

lmax = 8
Nmax = 14
M = 32
Lambda = 32
Nmax2 = 14
lmax2 = 15


## Analytic Integrals


In [8]:
u, sigma = smp.symbols("u sigma")

PhiHM = np.zeros((Nmax, lmax + 1))
for l1 in range(0, lmax2 + 1, 2):  # we only need even angular momenta
    for m1 in range(1, Nmax2 + 1):
        Fba = Fb(m1, l1, u, sigma)
        Fba = smp.expand(Fba)
        Fba = smp.simplify(Fba)

        numer, denom = smp.fraction(Fba)
        integral = smp.integrate(smp.Mul(numer, smp.Pow(denom, -1)), (sigma, 1, u / 2))
        result = smp.simplify(integral)

        PhiHM[m1, l1] = smp.lambdify(u, result)

print("PhiHM =", PhiHM)

TypeError: loop of ufunc does not support argument 0 of type Mul which has no callable sqrt method

Now, we will take our initial values, defined by $s_0$, $t_0$, and $u_0$, such that
$$
    s_0 = t_0 = u_0 = \frac{4}{3}.
$$

In addition to that, we have
$$
\begin{align*}
    A_{n, \ell} = 2 \delta_{\ell_1, 0} \Phi_V(m_1, j_1) + 2 \hat{\Phi}_{n \ell V}(m_1, \ell_1, j_2)
\end{align*}
$$

In [5]:
s0, t0, u0 = 4 / 3, 4 / 3, 4 / 3


Anl = np.ones((Nmax2 + 1, lmax, Lambda))
Anml = np.ones((Nmax + 1, Nmax2 + 1, lmax, Lambda))

for j1 in range(1, Lambda + 1):
    for l in range(1, lmax + 1):
        l1 = 2 * (l - 1)
        for m1 in range(1, Nmax2 + 1):
            for m1 in range(0, Nmax + 1):
                Anl[m1, l, j1] = (
                    2 * delta(l1, 0) * PHiV[m1, j1] + 2 * PhiHatnlV[m1, l1, j1]
                )

We will now define the Lambda function with the following argument.
$$
    \Lambda(\ell, s) \equiv \left( \frac{\sqrt{s} - 2}{\sqrt{s} + 2} \right)^{\left( \frac{\ell}{2} \right)}.
$$

In [6]:
def Lambda(l, s):
    return ((np.sqrt(s) - 2) / (np.sqrt(s) + 2)) ** (l / 2)


LambdaMatrix = np.ones((lmax + 1, M))
print(np.shape(LambdaMatrix))
for l in range(0, lmax + 1):
    for j1 in range(1, M + 1):
        LambdaMatrix[l, j1 - 1] = Lambda(2 * l, s[j1 - 1])
        np.nan_to_num(LambdaMatrix, copy=False)

print(LambdaMatrix)

(21, 40)
[[1.0000000000000000e+00 1.0000000000000000e+00 1.0000000000000000e+00
  1.0000000000000000e+00 1.0000000000000000e+00 1.0000000000000000e+00
  1.0000000000000000e+00 1.0000000000000000e+00 1.0000000000000000e+00
  1.0000000000000000e+00 1.0000000000000000e+00 1.0000000000000000e+00
  1.0000000000000000e+00 1.0000000000000000e+00 1.0000000000000000e+00
  1.0000000000000000e+00 1.0000000000000000e+00 1.0000000000000000e+00
  1.0000000000000000e+00 1.0000000000000000e+00 1.0000000000000000e+00
  1.0000000000000000e+00 1.0000000000000000e+00 1.0000000000000000e+00
  1.0000000000000000e+00 1.0000000000000000e+00 1.0000000000000000e+00
  1.0000000000000000e+00 1.0000000000000000e+00 1.0000000000000000e+00
  1.0000000000000000e+00 1.0000000000000000e+00 1.0000000000000000e+00
  1.0000000000000000e+00 1.0000000000000000e+00 1.0000000000000000e+00
  1.0000000000000000e+00 1.0000000000000000e+00 1.0000000000000000e+00
  1.0000000000000000e+00]
 [6.4263495147059236e-05 5.789664910967112

We will now define the Legendre function of the second kind $Q_n$ with the following argument.
$$
    Q(\ell, x, s) \equiv Q_n \left( \ell, 1 + \frac{2x}{s - 4} \right).
$$

In [7]:
def LegendreQ(l, x, s):
    return lqn(l, 1 + 2 * x / (s - 4))


legendreQMatrix = np.zeros((lmax + 1, M, M))
print(np.shape(legendreQMatrix))
for l in range(0, lmax + 1):
    for j1 in range(1, M + 1):
        for j in range(1, M + 1):
            legendreQMatrix[l, j1 - 1, j - 1] = LegendreQ(
                2 * (l + 1), s[j1 - 1], s[j - 1]
            )[0][2 * l]
            np.nan_to_num(legendreQMatrix, copy=False)  # Replace NaN with 0

            # print("l:", l)
            # print("j1:", j1)
            # print("j:", j)
            # print("leg:", LegendreQ(2*(l + 1), s[j1 - 1], s[j - 1])[0][2*l])

print(legendreQMatrix)

(21, 40, 40)
[[[1.2849396058674177e-004 1.1576358427727879e-003
   3.2222823188321815e-003 ... 2.1221497663468822e+000
   2.6304152838256267e+000 3.7277435398989076e+000]
  [1.2822972166975200e-004 1.1552576881221203e-003
   3.2156763344493577e-003 ... 2.1211351428521956e+000
   2.6293912268188486e+000 3.7267147296549159e+000]
  [1.2770124782305879e-004 1.1505013845432713e-003
   3.2024642140029511e-003 ... 2.1190997151340323e+000
   2.6273368162527722e+000 3.7246507538460127e+000]
  ...
  [1.8435862277053041e-006 1.6626212650360516e-005
   4.6373444363436643e-005 ... 3.4297513626034387e-001
   6.6077866653612238e-001 1.6250762971791324e+000]
  [6.6709721108135569e-007 6.0162115305841760e-006
   1.6780595362072039e-005 ... 1.5251197256869792e-001
   3.4527448802683686e-001 1.1498774480711040e+000]
  [7.4312462818785860e-008 6.7018961154751719e-007
   1.8693305917668197e-006 ... 1.9480754353767959e-002
   5.2548599213080020e-002 3.4642904153921633e-001]]

 [[2.8286999482026739e-013 2.06

With that definition, we can now rewrite $\hat{\mathbf{\Phi}}$ as
$$
    \hat{\mathbf{\Phi}} (\ell, x, s) = \frac{1}{\pi} \sqrt{\frac{x - 4}{4 - sa}} \left[ -2 \delta_{\ell 0} + \frac{4 (x - sa)}{s - 4} Q(\ell, x, s) \right].
$$

In [8]:
def PhiHat(l, x, s, sa, Q):
    # if (l == 0):
    return (
        1
        / np.pi
        * np.sqrt((x - 4) / (4 - sa))
        * ((l == 0) * (-2) + 4 * (x - sa) / (s - 4) * Q)
    )
    # else:
    #     return 1/np.pi * np.sqrt((x - 4)/(4 - sa)) * (4*(x - sa)/(s - 4) * Q)


PhiHatMatrix = np.zeros((lmax + 1, M, M))
print(np.shape(PhiHatMatrix))
for l in range(0, lmax + 1):
    for j1 in range(1, M + 1):
        for j in range(1, M + 1):
            PhiHatMatrix[l, j1 - 1, j - 1] = PhiHat(
                l, s[j1 - 1], s[j - 1], sa, legendreQMatrix[l, j1 - 1, j - 1]
            )
            np.nan_to_num(PhiHatMatrix, copy=False)  # Replace NaN with 0

            # print("l:", l)
            # print("j1:", j1)
            # print("j:", j)
            # print("PhiHat:", PhiHat(l, s[j1 - 1], s[j - 1]) * legendreQMatrix[l, j1 - 1, j - 1])

print(PhiHatMatrix)

(21, 40, 40)
[[[-4.1672021636944443e-003 -4.1757768729767872e-003
   -4.1929615572138439e-003 ... -1.1986691145258159e-002
   -1.2272785934940657e-002 -1.2465649855621175e-002]
  [-1.2488742230195356e-002 -1.2514466387480444e-002
   -1.2566020935432918e-002 ... -3.5993057614716224e-002
   -3.6854408056544115e-002 -3.7435147313595166e-002]
  [-2.0771583911245329e-002 -2.0814457114041299e-002
   -2.0900382027815557e-002 ... -6.0098725125784978e-002
   -6.1544598386158313e-002 -6.2519706065666814e-002]
  ...
  [-3.0913009294493293e-002 -3.1008102504645352e-002
   -3.1199456712452181e-002 ... -1.9869158942293044e+000
   -3.3714813689922600e+000 -5.6204657069504025e+000]
  [-1.8681486294821040e-002 -1.8739130446112463e-002
   -1.8855131039889356e-002 ... -1.5787832030504738e+000
   -3.3146260842766426e+000 -8.0327362110490856e+000]
  [-6.2495982963098390e-003 -6.2689120790477715e-003
   -6.3077790410491722e-003 ... -6.3356444823838054e-001
   -1.6796430296801785e+000 -9.9484527918892454e+00

Additionally, we can rewrite $\tilde{\mathbf{\Phi}}$ as
$$
    \tilde{\mathbf{\Phi}} (\ell, x_1, x_2, s) = \frac{1}{\pi^2} \frac{\sqrt{(x_1 - 4)(x_2 - 4)}}{4 - sa} \left[ 2 \delta_{\ell 0} - \frac{4 (x_1 - sa) (s - 4 + sa + x_1)}{(s - 4 + x_1 + x_2) (s - 4)} Q(\ell, x_1, s) - \frac{4 (x_2 - sa) (s - 4 + sa + x_2)}{(s - 4 + x_1 + x_2) (s - 4)} (-1)^{\ell} Q(\ell, x_2, s) \right]
$$

In [9]:
def PhiTilde(l, x1, x2, s, sa, Q1, Q2):
    return (
        1
        / (np.pi) ** 2
        * np.sqrt((x1 - 4) * (x2 - 4))
        / (4 - sa)
        * (
            (l == 0) * 2
            - 4 * (x1 - sa) * (s - 4 + sa + x1) / (s - 4 + x1 + x2) / (s - 4) * Q1
            - 4
            * (x2 - sa)
            * (s - 4 + sa + x2)
            / (s - 4 + x1 + x2)
            / (s - 4)
            * (-1) ** l
            * Q2
        )
    )


PhiTildeMatrix = np.zeros((lmax + 1, M, M, M))
print(np.shape(PhiTildeMatrix))
for l in range(0, lmax + 1):
    for j in range(1, M + 1):
        for j1 in range(1, M + 1):
            for j2 in range(j1, M + 1):
                PhiTildeMatrix[l, j1 - 1, j2 - 1, j - 1] = PhiTilde(
                    l,
                    s[j1 - 1],
                    s[j2 - 1],
                    s[j - 1],
                    sa,
                    legendreQMatrix[l, j1 - 1, j - 1],
                    legendreQMatrix[l, j2 - 1, j - 1],
                )
                PhiTildeMatrix[l, j2 - 1, j1 - 1, j - 1] = PhiTildeMatrix[
                    l, j1 - 1, j2 - 1, j - 1
                ]
                np.nan_to_num(PhiTildeMatrix, copy=False)  # Replace NaN with 0

                # print("l:", l)
                # print("j:", j)
                # print("j1:", j1)
                # print("j2:", j2)
                # print("PhiTilde:", PhiTilde(l, xi[j1 - 1], xi[j2 - 1]))

# print(PhiTildeMatrix)

(21, 40, 40, 40)


We now convert the coefficients of the partial waves from a matrix form into a linear form to easily store in a data file.

In [10]:
Delta = np.pi / M


def delta(x, y):
    return 1 if x == y else 0


Mfn = int(M * (M + 1) / 2)
Mfl = (lmax + 1) * M  # Number of interpolation points in the partial waves
A0h = 1  # f0    -> functional c0
B0h = 0  # f0    -> functional c2
hl0_RM = np.zeros((Mfl))  # f0    -> Re(h)
A1h = np.zeros(M)  # sigma -> functional c0
B1h = np.zeros(M)  # sigma -> functional c2
hl1_RM = np.zeros((Mfl, M))  # sigma -> Re(h)
hl1_IM = np.zeros((Mfl, M))  # sigma -> Im(h)
A2h = np.zeros(Mfn)  # rho   -> functional c0
B2h = np.zeros(Mfn)  # rho   -> functional c2
hl2_RM = np.zeros((Mfl, Mfn))  # rho   -> Re(h)
hl2_IM = np.zeros((Mfl, Mfn))  # rho   -> Im(h)

print("Mfn:", Mfn)
print("Mfl:", Mfl)
print("Saving coefficients...")

# Now we put computation data in matrix form
##### F0 #####
for j in range(1, M + 1):
    jf1 = j - 1  # only l = 0
    hl0_RM[jf1] = 2 * rho[j - 1]

##### SIGMA #####
for j in range(1, M + 1):
    A1h[j - 1] = a1[j - 1] * Delta
    B1h[j - 1] = b1[j - 1] * Delta

for j in range(1, M + 1):
    hl1_IM[j - 1, j - 1] = 2 * rho[j - 1]

for j1 in range(1, M + 1):  # l = 0 part
    for j in range(1, M + 1):
        hl1_RM[j - 1, j1 - 1] = 2 * rho[j - 1] * KMatrix_i[j - 1, j1 - 1]

for l in range(0, lmax + 1):
    for j in range(1, M + 1):
        for j1 in range(1, M + 1):
            jf1 = l * M + j - 1
            jf2 = j1 - 1
            hl1_RM[jf1, jf2] += 2 * rho[j - 1] * Delta * PhiHatMatrix[l, j1 - 1, j - 1]

##### RHO #####
jf2 = 0
for j1 in range(1, M + 1):
    for j2 in range(j1, M + 1):
        A2h[jf2] = a2[j1 - 1, j2 - 1] * Delta**2
        B2h[jf2] = b2[j1 - 1, j2 - 1] * Delta**2
        jf2 += 1

for l in range(0, lmax + 1):
    for j in range(1, M + 1):
        jf2 = 0
        for j1 in range(1, M + 1):
            for j2 in range(j1, M + 1):
                jf1 = l * M + j - 1
                hl2_RM[jf1, jf2] = rho[j - 1] * (
                    KMatrix_i[j - 1, j1 - 1] * Delta * PhiHatMatrix[l, j2 - 1, j - 1]
                    + KMatrix_i[j - 1, j2 - 1] * Delta * PhiHatMatrix[l, j1 - 1, j - 1]
                    + Delta**2 * PhiTildeMatrix[l, j1 - 1, j2 - 1, j - 1]
                )
                jf2 += 1

for l in range(0, lmax + 1):
    for j in range(1, M + 1):
        jf2 = 0
        for j1 in range(1, M + 1):
            for j2 in range(j1, M + 1):
                jf1 = l * M + j - 1
                hl2_IM[jf1, jf2] = rho[j - 1] * (
                    delta(j - 1, j1 - 1) * Delta * PhiHatMatrix[l, j2 - 1, j - 1]
                    + delta(j - 1, j2 - 1) * Delta * PhiHatMatrix[l, j1 - 1, j - 1]
                )
                jf2 += 1

print("Done")

Mfn: 820
Mfl: 840
Saving coefficients...
Done


TBD

In [11]:
hl0_RMA = np.zeros(Mfl)  # f0    -> Re(h/Lambda)
hl1_IMA = np.zeros((Mfl, M))  # sigma -> Im(h/Lambda)
hl1_RMA = np.zeros((Mfl, M))  # sigma -> Re(h/Lambda)
hl2_IMA = np.zeros((Mfl, Mfn))  # rho   -> Im(h/Lambda)
hl2_RMA = np.zeros((Mfl, Mfn))  # rho   -> Re(h/Lambda)
hl1_IMB = np.zeros((Mfl, M))  # sigma -> Im(h/Lambda^2)
hl2_IMB = np.zeros((Mfl, Mfn))  # rho   -> Im(h/Lambda^2)


LambdaV = np.ones(Mfl)
for l in range(0, lmax + 1):
    for j in range(1, M + 1):
        jf1 = l * M + j - 1
        LambdaV[jf1] = LambdaMatrix[l, j - 1]

for jf1 in range(1, Mfl + 1):
    hl0_RMA[jf1 - 1] = hl0_RM[jf1 - 1] / LambdaV[jf1 - 1]

for jf1 in range(1, Mfl + 1):
    for jf2 in range(0, M):
        hl1_RMA[jf1 - 1, jf2 - 1] = hl1_RM[jf1 - 1, jf2 - 1] / LambdaV[jf1 - 1]
        hl1_IMA[jf1 - 1, jf2 - 1] = hl1_IM[jf1 - 1, jf2 - 1] / LambdaV[jf1 - 1]
        hl1_IMB[jf1 - 1, jf2 - 1] = hl1_IM[jf1 - 1, jf2 - 1] / LambdaV[jf1 - 1] ** 2

for jf1 in range(1, Mfl + 1):
    for jf2 in range(0, Mfn):
        hl2_RMA[jf1 - 1, jf2 - 1] = hl2_RM[jf1 - 1, jf2 - 1] / LambdaV[jf1 - 1]
        hl2_IMA[jf1 - 1, jf2 - 1] = hl2_IM[jf1 - 1, jf2 - 1] / LambdaV[jf1 - 1]
        hl2_IMB[jf1 - 1, jf2 - 1] = hl2_IM[jf1 - 1, jf2 - 1] / LambdaV[jf1 - 1] ** 2

Save the data in a text file.

In [12]:
saf = str(sa).replace(".", "p")
dataKFile = os.path.join(
    os.getcwd(), f"primal_3.0_points_data_P2_nopole_sa{saf}_lmax{lmax}_M{M}.dat"
)

with open(dataKFile, "w") as fd:
    for j in range(1, M + 1):
        fd.write(f"{xi[j - 1]}\n")

    fd.write(f"{A0h}\n")

    fd.write(f"{B0h}\n")

    for jf1 in range(1, Mfl + 1):
        fd.write(f"{hl0_RM[jf1 - 1]}\n")

    for j in range(1, M + 1):
        fd.write(f"{A1h[j - 1]}\n")

    for j in range(1, M + 1):
        fd.write(f"{B1h[j - 1]}\n")

    for jf2 in range(1, M + 1):
        for jf1 in range(1, Mfl + 1):
            fd.write(f"{hl1_RM[jf1 - 1, jf2 - 1]}\n")

    for jf2 in range(1, M + 1):
        for jf1 in range(1, Mfl + 1):
            fd.write(f"{hl1_IM[jf1 - 1, jf2 - 1]}\n")

    for j in range(1, Mfn + 1):
        fd.write(f"{A2h[j - 1]}\n")

    for j in range(1, Mfn + 1):
        fd.write(f"{B2h[j - 1]}\n")

    for jf2 in range(1, Mfn + 1):
        for jf1 in range(1, Mfl + 1):
            fd.write(f"{hl2_RM[jf1 - 1, jf2 - 1]}\n")

    for jf2 in range(1, Mfn + 1):
        for jf1 in range(1, Mfl + 1):
            fd.write(f"{hl2_IM[jf1 - 1, jf2 - 1]}\n")

    ##### NOW RESCALED #####
    for jf1 in range(1, Mfl + 1):
        fd.write(f"{hl0_RMA[jf1 - 1]}\n")

    for jf2 in range(1, M + 1):
        for jf1 in range(1, Mfl + 1):
            fd.write(f"{hl1_RMA[jf1 - 1, jf2 - 1]}\n")

    for jf2 in range(1, M + 1):
        for jf1 in range(1, Mfl + 1):
            fd.write(f"{hl1_IMA[jf1 - 1, jf2 - 1]}\n")

    for jf2 in range(1, Mfn + 1):
        for jf1 in range(1, Mfl + 1):
            fd.write(f"{hl2_RMA[jf1 - 1, jf2 - 1]}\n")

    for jf2 in range(1, Mfn + 1):
        for jf1 in range(1, Mfl + 1):
            fd.write(f"{hl2_IMA[jf1 - 1, jf2 - 1]}\n")

    for jf2 in range(1, M + 1):
        for jf1 in range(1, Mfl + 1):
            fd.write(f"{hl1_IMB[jf1 - 1, jf2 - 1]}\n")

    for jf2 in range(1, Mfn + 1):
        for jf1 in range(1, Mfl + 1):
            fd.write(f"{hl2_IMB[jf1 - 1, jf2 - 1]}\n")

    for jf1 in range(1, Mfl + 1):
        fd.write(f"{LambdaV[jf1 - 1]}\n")

## Notes
A relation between the two solutions $P_n(x)$ and $Q_n(x)$ of the Legendre differential equation is given by
$$
Q_n(x) = P_n(x) \int \frac{1}{P_n^2(x) (1 - x^2)} ~ \mathrm{d} x.
$$